# Hidden Markov Models (HMMs) & NLTK
Using manually tagging corpus Bahasa Indonesia

In [1]:
from pprint import pprint
from nltk.tag import hmm
from sklearn.externals import joblib
from nltk.tag.hmm  import HiddenMarkovModelTagger, HiddenMarkovModelTrainer
from nltk.probability import LidstoneProbDist

## Load Corpus

In [2]:
fname = "Indonesian_Manually_Tagged_Corpus.tsv"
fopen = open(fname,'r')
fdata = fopen.readlines()

In [3]:
sentences = []
sentence = []
for word in fdata:
    dt = word.replace('\n','').split('\t')
    if(len(dt) == 1):
        sentences.append(sentence)
        sentence = []
    else:
        sentence.append((dt[0],dt[1]))

## Split Training and Test Data

In [4]:
print('All Sentences : %d ' % (len(sentences)))
cutoff = int(.999 * len(sentences))
training_sentences = sentences[:cutoff]
test_sentences = sentences[cutoff:]
print('Training Sentences : %d ' % (len(training_sentences)))
print('Testing Sentences : %d ' % (len(test_sentences)))

All Sentences : 10029 
Training Sentences : 10018 
Testing Sentences : 11 


## Training Model

In [5]:
print 'Training Start'
trainer = HiddenMarkovModelTrainer()
tagger = trainer.train_supervised(training_sentences, estimator=lambda fd, bins: LidstoneProbDist(fd, 0.1, bins))
print 'Training Completed'

print 'Testing Start'
tagger.test(test_sentences, verbose=True)
print 'Testing Completed'

Training Start
Training Completed
Testing Start
Test: Jam/NN malam/NN akan/MD berlaku/VB efektif/JJ pada/IN pukul/NN 8/CD :/Z 30/CD sore/NN Kamis/NNP dan/CC akan/MD berakhir/VB sampai/IN pukul/NN 6/CD :/Z 0/CD pagi/NN waktu/NN setempat/NN Jumat/NNP ,/Z kata/VB seorang/NND juru bicara/NN militer/NN ./Z

Untagged: Jam malam akan berlaku efektif pada pukul 8 : 30 sore Kamis dan akan berakhir sampai pukul 6 : 0 pagi waktu setempat Jumat , kata seorang juru bicara militer .

HMM-tagged: Jam/DT malam/NN akan/MD berlaku/VB efektif/JJ pada/IN pukul/NN 8/CD :/Z 30/CD sore/NN Kamis/NNP dan/CC akan/MD berakhir/VB sampai/IN pukul/NN 6/CD :/Z 0/CD pagi/NN waktu/NN setempat/NN Jumat/NNP ,/Z kata/VB seorang/NND juru bicara/NN militer/NN ./Z

Entropy: 10.7773913296

------------------------------------------------------------
Test: Dua/CD mahasiswa/NN oposisi/NN dan/CC dua/CD orang/NN lain/JJ -nya/PRP tewas/VB ditembak/VB dan/CC 100/CD orang/NN lain/JJ -nya/PRP cedera/JJ ,/Z kebanyakan/CD karena/SC se

## Save Model

In [6]:
import dill
with open('my_tagger.dill', 'wb') as f:
    dill.dump(tagger, f)

## Load Model

In [7]:
with open('my_tagger.dill', 'rb') as f:
    hmm_tagger = dill.load(f)

In [8]:
from nltk.tokenize import WordPunctTokenizer
from nltk import sent_tokenize
from nltk import word_tokenize, RegexpParser, ne_chunk
from collections import defaultdict
import string

In [9]:
def hasNumbers(inputString):
    result = False
    for char in list(inputString):
        if(char.isdigit()):
            result = True
    return result

def xcheck_tag(word,tag):
    punc = list(string.punctuation)
    punc.append('.')
    punc.append(',')
    punc.append('"')
    punc.append("'")
    
    dates = ['Januari','Februari','Maret','April','Mei','Juni','Juli','Agustus','September','Oktober','November','Desember',\
            'Jan','Feb','Mar','Apr','Mei','Jun','Jul','Agt','Sep','Okt','Nov','Des',\
            'januari','februari','maret','april','mei','juni','juli','agustus','september','oktober','november','desember',\
            'Senin','Selasa','Rabu','Kamis','Jumat','Sabtu','Minggu'
        ]
    
    if(word in dates):
        tag = 'DATE'
    
    if(word in punc):
        tag = 'Z'
        
    if(tag == 'CD' and word.isdigit()):
        tag = 'CD'
        
    if(tag in ['SYM','Z','CD','MD'] and word.upper() != word and hasNumbers(word) == False \
      and word[-3:] not in ['nya','kah','lah']):
        tag = 'NNP'
    
    if(tag == 'NN' and word[:1].upper() == word):
        tag = 'NNP'
        
    if(tag == 'NNP' and word.lower() == word):
        tag = 'NN'
    
    if(tag == 'NNP' and len(word) == 1):
        tag = 'NN'
        
    if(tag == 'FW' and word.lower() == word):
        tag = 'NN'
        
    return word,tag

def get_entity(wording,verbose=False):
    word_punct_tokenizer = WordPunctTokenizer()
    toktok = word_punct_tokenizer.tokenize(wording)
    final_tagged = []
    pos_tagging_begin = hmm_tagger.tag(toktok)
    for ptb in pos_tagging_begin:
        w,t = xcheck_tag(ptb[0],ptb[1])
        final_tagged.append((w,t))
    
    if(verbose):
        print(pos_tagging_begin)
    
    grammar = """
    ENTITY : {<NNP>+}
    ENTITY : {<FW>+}
    """
    result = []
    chunkParser = RegexpParser(grammar)
    tree = chunkParser.parse(final_tagged)
    for subtree in tree.subtrees():
        if(subtree.label()=="ENTITY"):
            tampung_entity = []
            for se in subtree.leaves():
                tampung_entity.append(se[0])
            result.append(' '.join(tampung_entity))
    return result

def leaders(xsz, top=10):
    counts = defaultdict(int)
    for xs in xsz:
        for x in xs:
            counts[x] += 1
    return sorted(counts.items(), reverse=True, key=lambda tup: tup[1])[:top]

In [10]:
wording = '''Komisi Pemberantasan Korupsi melakukan operasi tangkap tangan terhadap istri Gubernur Bengkulu, Lili Martiani Maddari, dan pengusaha Rico Diansari pada Selasa (20/6/2017). Suap diduga diberikan untuk memuluskan proyek jalan di provinsi tersebut.
Rico Diansari yang merupakan Bendahara DPD Golkar Provinsi Bengkulu langsung ditangkap KPK setelah menyerahkan uang dalam kardus dengan nominal diperkirakan Rp1 miliar. Uang diduga diterima Lili Martiani, di kediamannya di kawasan Sidomulyo, Bengkulu.
Tak lama, KPK mengamankan Gubernur Bengkulu Ridwan Mukti dan menetapkannya sebagai tersangka. Suami istri itu pun kini mendekam di tahanan KPK.
Ridwan diduga menerima suap dari pihak swasta terkait proyek pembangunan jalan yang nilai proyeknya diperkirakan mencapai Rp90 miliar. Penerimaan tersebut diduga merupakan yang pertama dari total komitmen yang disepakati.
Ridwan menyatakan permohonan maaf kepada masyarakat. Ia pun menyatakan mundur dari kursi Gubernur Bengkulu. Ridwan juga menyatakan mundur sebagai Ketua DPD Golkar Provinsi Bengkulu.
"Ya mohon maaf lah, saya harus bertanggung jawab terhadap kekhilafan istri," kata Ridwan Mukti dengan rompi tahanan saat keluar dari Gedung KPK, Rabu (21/6/2017).
Kompak pasutri terjerat korupsi
Ridwan-Lili bukanlah pasangan suami istri pertama yang terjerat kasus korupsi. Ada sejumlah pasangan suami istri yang kompak terjerat korupsi dan telah menjalani masa tahanan, di antaranya mantan Gubernur Sumatera Utara, Gatot Pujo Nugroho dan istrinya Evy Susanti.
Gatot dan Evy telah terbukti memberikan suap kepada Hakim serta Panitera PTUN Medan. Keduanya juga terbukti telah memberikan suap Rp200 juta kepada Patrice Rio Capella mantan Sekretaris Jenderal Partai NasDem dan anggota Komisi lll Dewan Perwakilan Rakyat.
Gatot Pujo Nugroho dihukum penjara tiga tahun dalam kasus suap terhadap hakim di Medan, Senin (14/3/2016). Adapun istri muda Gatot, Evy Susanti menjalani hukuman 2,5 tahun penjara.
Sebelum Gatot-Evy pasangan suami istri M. Nazaruddin dan istrinya, Neneng Sri Wahyuni telah lebih dulu menjadi terpidana korupsi. Nazaruddin dan Neneng kini masih menjalani masa hukumannya. Putusan berkekuatan hukum mengirim Neneng 6 tahun ke penjara, sedang Nazaruddin 7 tahun.
Pasutri lainnya adalah wali kota Palembang, Sumatera Selatan, Tomi Herton dan istrinya, Masyito. Kasus yang menjeratnya adalah penyuapan terhadap mantan ketua MK, Akil Mochtar.
Pengadilan Tipikor menjatuhkan hukuman 6 tahun penjara kepada Romi Herton, dan 4 tahun penjara kepada Masyito. Pengadilan tinggi DKI menambah hukuman Romi menjadi 7 tahun dan istrinya 5 tahun.
Ada juga Bupati Karawang Ade Swara dan istrinya, Nurlatifah. Keduanya terjaring dalam operasi tangkap tangan, dalam kasus pemerasan pembangunan mal di Karawang. April lalu Pengadilan Tipikor menjatuhkan hukuman 6 tahun penjara untuk Ade, sedang Nurlatifah 5 tahun.
'''
senttok = sent_tokenize(wording)

In [11]:
all_entity = []
for xsent in senttok:
    ent = get_entity(xsent, verbose=False)
    all_entity.append(ent)
    print('Kalimat : %s' % xsent)
    print('Entity : ')
    print(ent)
    print('-------------------------------------------')
    print('')

Kalimat : Komisi Pemberantasan Korupsi melakukan operasi tangkap tangan terhadap istri Gubernur Bengkulu, Lili Martiani Maddari, dan pengusaha Rico Diansari pada Selasa (20/6/2017).
Entity : 
['Komisi Pemberantasan Korupsi', 'Gubernur Bengkulu', 'Lili Martiani Maddari', 'Rico Diansari']
-------------------------------------------

Kalimat : Suap diduga diberikan untuk memuluskan proyek jalan di provinsi tersebut.
Entity : 
[]
-------------------------------------------

Kalimat : Rico Diansari yang merupakan Bendahara DPD Golkar Provinsi Bengkulu langsung ditangkap KPK setelah menyerahkan uang dalam kardus dengan nominal diperkirakan Rp1 miliar.
Entity : 
['Rico Diansari', 'DPD Golkar Provinsi Bengkulu', 'KPK']
-------------------------------------------

Kalimat : Uang diduga diterima Lili Martiani, di kediamannya di kawasan Sidomulyo, Bengkulu.
Entity : 
['Lili Martiani', 'Bengkulu']
-------------------------------------------

Kalimat : Tak lama, KPK mengamankan Gubernur Bengkulu Ri

## List Entity

In [12]:
pprint(all_entity)

[['Komisi Pemberantasan Korupsi',
  'Gubernur Bengkulu',
  'Lili Martiani Maddari',
  'Rico Diansari'],
 [],
 ['Rico Diansari', 'DPD Golkar Provinsi Bengkulu', 'KPK'],
 ['Lili Martiani', 'Bengkulu'],
 ['KPK', 'Gubernur Bengkulu Ridwan Mukti'],
 ['KPK'],
 ['Ridwan'],
 [],
 ['Ridwan'],
 ['Gubernur Bengkulu'],
 ['Ridwan', 'Ketua DPD Golkar Provinsi Bengkulu'],
 ['Ridwan Mukti', 'Gedung KPK'],
 ['Ridwan'],
 ['Gubernur Sumatera Utara', 'Gatot Pujo Nugroho', 'Evy Susanti'],
 ['Gatot', 'Panitera PTUN Medan'],
 ['Patrice Rio Capella',
  'Sekretaris Jenderal Partai NasDem',
  'Komisi',
  'Dewan Perwakilan Rakyat'],
 ['Gatot Pujo', 'Medan'],
 ['Gatot', 'Susanti'],
 ['Gatot', 'Nazaruddin', 'Neneng Sri Wahyuni'],
 [],
 ['Neneng'],
 ['Palembang', 'Sumatera Selatan', 'Tomi Herton', 'Masyito'],
 ['MK', 'Akil Mochtar'],
 ['Pengadilan Tipikor', 'Romi Herton', 'Masyito'],
 ['DKI'],
 ['Bupati Karawang Ade Swara', 'Nurlatifah'],
 ['Karawang'],
 ['Pengadilan Tipikor', 'Ade']]


## Sorting Entity

In [13]:
d = leaders(all_entity,100)
pprint(d)

[('Ridwan', 4),
 ('KPK', 3),
 ('Gatot', 3),
 ('Pengadilan Tipikor', 2),
 ('Masyito', 2),
 ('Rico Diansari', 2),
 ('Gubernur Bengkulu', 2),
 ('Ridwan Mukti', 1),
 ('Ade', 1),
 ('Karawang', 1),
 ('Ketua DPD Golkar Provinsi Bengkulu', 1),
 ('Romi Herton', 1),
 ('Bupati Karawang Ade Swara', 1),
 ('Nurlatifah', 1),
 ('Gubernur Sumatera Utara', 1),
 ('Sumatera Selatan', 1),
 ('Nazaruddin', 1),
 ('Sekretaris Jenderal Partai NasDem', 1),
 ('Panitera PTUN Medan', 1),
 ('Lili Martiani Maddari', 1),
 ('Neneng Sri Wahyuni', 1),
 ('DKI', 1),
 ('Medan', 1),
 ('Lili Martiani', 1),
 ('Bengkulu', 1),
 ('Gatot Pujo', 1),
 ('Evy Susanti', 1),
 ('Patrice Rio Capella', 1),
 ('DPD Golkar Provinsi Bengkulu', 1),
 ('Dewan Perwakilan Rakyat', 1),
 ('Komisi Pemberantasan Korupsi', 1),
 ('Susanti', 1),
 ('Gedung KPK', 1),
 ('Palembang', 1),
 ('Neneng', 1),
 ('Akil Mochtar', 1),
 ('Tomi Herton', 1),
 ('Gubernur Bengkulu Ridwan Mukti', 1),
 ('MK', 1),
 ('Gatot Pujo Nugroho', 1),
 ('Komisi', 1)]
